In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
accidentdf = pd.read_csv("../input/road-traffic-collision-dataset/accident_data.csv",parse_dates=['Date'])

In [ ]:
accidentdf.sample(6)

In [ ]:
!pip install missingno

In [ ]:
import missingno as mno
# extract a list of null columns
null_columns = pd.DataFrame(accidentdf[accidentdf.columns[accidentdf.isnull().any()]].isnull().sum()*100/ accidentdf.shape[0], columns =["Percentage of null values"])
null_columns["Total Nan Values"]=accidentdf[accidentdf.columns[accidentdf.isnull().any()]].isnull().sum()
null_columns

First road class, second road class, lsoa of accident have a lot of missing values.The rest have less than 5% missing values

In [ ]:
# make a list of colums that have null values
null_cols = null_columns.index.tolist()
print(type(null_cols))
print(null_cols)

In [ ]:
# where exactly are the values missing?
mno.matrix(accidentdf[null_cols], figsize=(30,30))

In [ ]:
# shape of the data 
print("No of rows in the acident data is",accidentdf.shape[0])
print("No of columns in the accident data is", accidentdf.shape[1])

In [ ]:
accidentdf.info()

In [ ]:
accidentdf.describe().T.style

In [ ]:
# find duplicate entries

print("Duplicate entries:", accidentdf.duplicated().sum())
accidentdf = accidentdf.drop_duplicates(keep=False)
print("Duplicate entries:", accidentdf.duplicated().sum())

In [ ]:
# how many unique values to know what to datatypes to change
# to see all the columns
pd.set_option('Display.max_rows', None)
# nunique counts the unique values in every column and dropna=true means we ignore nan values and .sort sorts the values in ascending order
print(accidentdf.nunique(dropna=True).sort_values())


In [ ]:
# create a dataframe with null columns
nullcolumns=accidentdf[null_cols]
nullcolumns.head()

In [ ]:
# categorical features
categoricalfeatures = accidentdf.select_dtypes(include='object').columns
numericalfeatures = accidentdf.select_dtypes('number').columns
cat=accidentdf[categoricalfeatures].copy()
num=accidentdf[numericalfeatures].copy()
print(cat.sample(9))
print(num.sample(7))

# **Complete Case Analysis**

In [ ]:
# handling missing values
cols = [var for var in accidentdf.columns if accidentdf[var].isnull().mean() < 0.05 and accidentdf[var].isnull().mean()>0]
cols

In [ ]:
accidentdf[cols].sample(7)

In [ ]:
# see how much you lose when you drop them all
len(accidentdf[cols].dropna())/len(accidentdf)

In [ ]:
df = accidentdf[cols].dropna()
accidentdf.shape, df.shape

In [ ]:
# visualize the distribution 
df.hist(bins=50,density = True, figsize = (12,12))
plt.show()

In [ ]:
# check for changes in distribution
fig = plt.figure()
ax = fig.add_subplot(111)
# original data
accidentdf['Latitude'].hist(bins=50,ax=ax,density = True, color='red')
# data after cca
df['Latitude'].hist(bins=50,ax=ax,color='green',density=True,alpha=0.8)
plt.show

In [ ]:
# density plot
fig=plt.figure()
ax = fig.add_subplot(111)
accidentdf['Latitude'].plot.density(color='red')
df['Latitude'].plot.density(color='green')

In [ ]:
# percentage change for categorical variables
temp =pd.concat([
    accidentdf['Light_Conditions'].value_counts()/len(accidentdf),
    df['Light_Conditions'].value_counts()/len(df)
],
axis=1)
temp.columns=['original','cca']
temp

# **Findings**
**CCA** changes the distribution of:
*  Did_Police_Officer_Attend_Scene_of_Accident',
*  Latitude
*  Light_Conditions
*  Location_Easting_OSGR
*  Location_Northing_OSGR
*  Longitude

# **Mean/Median Imputation**

In [ ]:
accidentdf.isnull().mean()

In [ ]:
nullnumcols = [var for var in accidentdf.select_dtypes('number').columns if accidentdf[var].isnull().mean() < 0.05 and accidentdf[var].isnull().mean()>0]
numimpute=accidentdf[nullnumcols].copy()

In [ ]:
# mean impuation on where cca did not work
X = numimpute
y = accidentdf['Accident_Severity']

In [ ]:
XTrain,XTest,YTrain,YTest = train_test_split(X,y,test_size=0.2,random_state=22)

In [ ]:
XTrain.shape

In [ ]:
XTrain.isnull().mean()

In [ ]:
mean_1st = XTrain['1st_Road_Number'].mean()
median_1st = XTrain['1st_Road_Number'].median()

mean_2nd = XTrain['2nd_Road_Number'].mean()
median_2nd = XTrain['2nd_Road_Number'].median()

mean_Did_Police = XTrain['Did_Police_Officer_Attend_Scene_of_Accident'].mean()
median_Did_Police = XTrain['Did_Police_Officer_Attend_Scene_of_Accident'].median()

mean_Latitude = XTrain['Latitude'].mean()
median_Latitude = XTrain['Latitude'].median()

mean_Easting = XTrain['Location_Easting_OSGR'].mean()
median_Easting = XTrain['Location_Easting_OSGR'].median()

mean_Northing = XTrain['Location_Northing_OSGR'].mean()
median_Northing = XTrain['Location_Northing_OSGR'].median()

mean_Longitude = XTrain['Longitude'].mean()
median_Longitude = XTrain['Longitude'].median()

mean_Human = XTrain['Pedestrian_Crossing-Human_Control'].mean()
median_Human = XTrain['Pedestrian_Crossing-Human_Control'].median()

mean_Physical = XTrain['Pedestrian_Crossing-Physical_Facilities'].mean()
median_Physical = XTrain['Pedestrian_Crossing-Physical_Facilities'].median()



In [ ]:
XTrain['1st_mean']=XTrain['1st_Road_Number'].fillna(mean_1st)
XTrain['1st_median']=XTrain['1st_Road_Number'].fillna(median_1st)

XTrain['mean_2']=XTrain['2nd_Road_Number'].fillna(mean_2nd)
XTrain['median_2']=XTrain['2nd_Road_Number'].fillna(median_2nd)

XTrain['mean_Did_Police_Attend']=XTrain['Did_Police_Officer_Attend_Scene_of_Accident'].fillna(mean_Did_Police)
XTrain['median_Did_Police_Attend']=XTrain['Did_Police_Officer_Attend_Scene_of_Accident'].fillna(median_Did_Police)

XTrain['mean_Latitude'] = XTrain['Latitude'].fillna(mean_Latitude)
XTrain['median_Latitude'] = XTrain['Latitude'].fillna(median_Latitude)

XTrain['mean_Easting'] = XTrain['Location_Easting_OSGR'].fillna(mean_Easting)
XTrain['median_Easting'] = XTrain['Location_Easting_OSGR'].fillna(median_Easting)

XTrain['mean_Northing'] = XTrain['Location_Northing_OSGR'].fillna(mean_Northing)
XTrain['median_Northing'] = XTrain['Location_Northing_OSGR'].fillna(median_Northing)

XTrain['mean_Longitude'] = XTrain['Longitude'].fillna(mean_Longitude)
XTrain['median_Longitude'] = XTrain['Longitude'].fillna(median_Longitude)

XTrain['mean_Human'] = XTrain['Pedestrian_Crossing-Human_Control'].fillna(mean_Human)
XTrain['median_Human'] = XTrain['Pedestrian_Crossing-Human_Control'].fillna(median_Human)

XTrain['mean_Physical'] = XTrain['Pedestrian_Crossing-Physical_Facilities'].fillna(mean_Physical)
XTrain['median_Physical'] = XTrain['Pedestrian_Crossing-Physical_Facilities'].fillna(median_Physical)


In [ ]:
# how will the data look like if it got replaced by the mean?
XTrain.sample(10)

In [ ]:
print("Variance of 1st Road Number is:", XTrain['1st_Road_Number'].var())
print("Variance of the mean of 1st Road Number is:", XTrain['1st_mean'].var())
print("Variance of median of 1st Road Number is:", XTrain['1st_median'].var())

print("Variance of 2nd Road Number is:", XTrain['2nd_Road_Number'].var())
print("Variance of the mean of 2nd Road Number is:", XTrain['mean_2'].var())
print("Variance of median of 2nd Road Number is:", XTrain['median_2'].var())

print("Variance of Did_Police_Officer_Attend_Scene_of_Accident is:", XTrain['Did_Police_Officer_Attend_Scene_of_Accident'].var())
print("Variance of the mean of Did_Police_Officer_Attend_Scene_of_Accident is:", XTrain['mean_Did_Police_Attend'].var())
print("Variance of median of Did_Police_Officer_Attend_Scene_of_Accident is:", XTrain['median_Did_Police_Attend'].var())

print("Variance of Latitude is:", XTrain['Latitude'].var())
print("Variance of the mean of Latitude is:", XTrain['mean_Latitude'].var())
print("Variance of median of Latitude is:", XTrain['median_Latitude'].var())

print("Variance of Location_Easting_OSGR is:", XTrain['Location_Easting_OSGR'].var())
print("Variance of the mean of Location_Easting_OSGR is:", XTrain['mean_Easting'].var())
print("Variance of median of Location_Easting_OSGR is:", XTrain['median_Easting'].var())

print("Variance of Location_Northing_OSGR is:", XTrain['Location_Northing_OSGR'].var())
print("Variance of the mean of Location_Northing_OSGR is:", XTrain['mean_Northing'].var())
print("Variance of median of Location_Northing_OSGR is:", XTrain['median_Northing'].var())

print("Variance of Longitude is:", XTrain['Longitude'].var())
print("Variance of the mean of Longitude is:", XTrain['mean_Longitude'].var())
print("Variance of median of Longitude is:", XTrain['median_Longitude'].var())

print("Variance of Pedestrian_Crossing-Human_Control is:", XTrain['Pedestrian_Crossing-Human_Control'].var())
print("Variance of the mean of Pedestrian_Crossing-Human_Control is:", XTrain['mean_Human'].var())
print("Variance of median of Pedestrian_Crossing-Human_Control is:", XTrain['median_Human'].var())

print("Variance of Pedestrian_Crossing-Physical_Facilities is:", XTrain['Pedestrian_Crossing-Physical_Facilities'].var())
print("Variance of the mean of Pedestrian_Crossing-Physical_Facilities is:", XTrain['mean_Physical'].var())
print("Variance of median of Pedestrian_Crossing-Physical_Facilities is:", XTrain['median_Physical'].var())

In [ ]:
# to check if the distribution changed
fig = plt.figure()
ax = fig.add_subplot(111)
XTrain["2nd_Road_Number"].plot(kind='kde',ax=ax)
XTrain['mean_2'].plot(kind='kde',ax=ax,color = 'red')
XTrain['median_2'].plot(kind='kde',ax=ax,color='green')

lines,labels = ax.get_legend_handles_labels()
ax.legend(lines,labels,loc='best')
plt.show()

In [ ]:
# ideally we would want the variance the variance not disturbed
# more change , change in covariance with other variables
XTrain.cov()

In [ ]:
XTrain.corr()

# FINDINGS
* Mean/Median imputation can work on all numerical variables except 1st Road Number.
* The distribution is symmetric; mean =median.
* Data is less/not skewed.
* Mean imputation changes the behaviour of:
    *  Pedestrian_Crossing-Human_Control
    *  Did_Police_Officer_Attend_Scene_of_Accident
    *  Pedestrian_Crossing-Physical_Facilities     

In [ ]:
# box plot to check for outliers
XTrain[['Pedestrian_Crossing-Physical_Facilities','mean_Physical', 'median_Physical']].boxplot()


In [ ]:
# Using Sklearn 
imputer1 = SimpleImputer(strategy='mean')
imputer2=SimpleImputer(strategy='median')

In [ ]:
transformer = ColumnTransformer([('imputer1', imputer1, ['Latitude']),
                                ('imputer2', imputer2, ['Pedestrian_Crossing-Physical_Facilities'])], remainder='passthrough')

In [ ]:
transformer.fit(XTrain)

In [ ]:
# fit transformer
# the mean of column Latitude
transformer.named_transformers_['imputer1'].statistics_